In [406]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

In [427]:
# Load the pretrained model
model = models.vgg16(pretrained=True)
model.eval();
# new_classifier = nn.Sequential(*list(model.classifier.children())[:-1])
# model.classifier = new_classifier
# model.classifier;

In [414]:
layer = list(model.classifier.children())[0]
layer

Linear (25088 -> 4096)

In [415]:
scaler = transforms.Scale(224)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [416]:
def get_vector(image_name):
    img = Image.open(image_name)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
#     t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
#     # 1. Load the image with Pillow library
#     img = image_loader(image_name)
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding

In [417]:
img1 = './Set1/image1.jpg'

In [418]:
model._modules;

In [419]:
fvex1 = get_vector(img1)

RuntimeError: size mismatch, m1: [1 x 32256], m2: [25088 x 4096] at /Users/soumith/code/builder/wheel/pytorch-src/torch/lib/TH/generic/THTensorMath.c:1237

In [ ]:
models.resnet18()._modules

In [420]:
model(img1)

TypeError: argument 0 is not a Variable

In [428]:
img = Image.open(img1)
# 2. Create a PyTorch Variable with the transformed image
t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))

In [429]:
model(t_img)

RuntimeError: size mismatch, m1: [1 x 32256], m2: [25088 x 4096] at /Users/soumith/code/builder/wheel/pytorch-src/torch/lib/TH/generic/THTensorMath.c:1237